In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Navigate to the directory containing the zip file
%cd /content/drive/MyDrive/

# Optional: Create a directory to extract the files into
!mkdir -p extracted_images

# Unzip the file into the specified directory
!unzip image-search.zip -d extracted_images

# List the contents of the directory to verify
%ls extracted_images/


Mounted at /content/drive
/content/drive/MyDrive
Archive:  image-search.zip
  inflating: extracted_images/queries/queries/0.jpg  
  inflating: extracted_images/queries/queries/1.jpg  
  inflating: extracted_images/queries/queries/10.jpg  
  inflating: extracted_images/queries/queries/11.jpg  
  inflating: extracted_images/queries/queries/12.jpg  
  inflating: extracted_images/queries/queries/13.jpg  
  inflating: extracted_images/queries/queries/14.jpg  
  inflating: extracted_images/queries/queries/15.jpg  
  inflating: extracted_images/queries/queries/16.jpg  
  inflating: extracted_images/queries/queries/17.jpg  
  inflating: extracted_images/queries/queries/18.jpg  
  inflating: extracted_images/queries/queries/19.jpg  
  inflating: extracted_images/queries/queries/2.jpg  
  inflating: extracted_images/queries/queries/20.jpg  
  inflating: extracted_images/queries/queries/21.jpg  
  inflating: extracted_images/queries/queries/3.jpg  
  inflating: extracted_images/queries/queries/4.

In [ ]:
from transformers import AutoModel, CLIPProcessor
import os
import pandas as pd
from PIL import Image
import torch

# Function to preprocess and extract features from images
def preprocess_and_extract_features(image_path, processor, model):
    image = Image.open(image_path).convert('RGB')
    inputs = processor(images=[image], return_tensors='pt').to('cuda')
    outputs = model.get_image_features(inputs.pixel_values).cpu()
    normalized_outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
    return normalized_outputs

if __name__ == '__main__':
    src_dir = '/content/drive/MyDrive/extracted_images/test/images'
    query_dir = '/content/drive/MyDrive/extracted_images/queries/queries'
    submission = pd.read_csv('/content/drive/MyDrive/extracted_images/sample_submission.csv')

    model_name = 'laion/CLIP-ViT-H-14-laion2B-s32B-b79K'
    model = AutoModel.from_pretrained(model_name).cuda().eval()
    processor = CLIPProcessor.from_pretrained(model_name)


    dot_product_threshold = 0.07
    cosine_similarity_threshold = 0.75

    submission['dot_class'] = 22
    submission['cosine_class'] = 22

    with torch.no_grad():
        query_images = [preprocess_and_extract_features(os.path.join(query_dir, file), processor, model) for file in os.listdir(query_dir)]
        query_classes = [int(file[:-4]) for file in os.listdir(query_dir)]
        query_images = torch.cat(query_images)

        for idx, row in submission.iterrows():
            if not pd.isna(row['class']):
                continue
            image_features = preprocess_and_extract_features(os.path.join(src_dir, row['img_file']), processor, model)
            dot_product_values = image_features @ query_images.T
            if dot_product_values.max() > dot_product_threshold:
                submission.at[idx, 'dot_class'] = query_classes[dot_product_values.argmax().numpy().tolist()]
            cosine_values = torch.cosine_similarity(image_features, query_images)
            if cosine_values.max() > cosine_similarity_threshold:
                submission.at[idx, 'cosine_class'] = query_classes[cosine_values.argmax().numpy().tolist()]

        # Save submissions based on enhanced metrics
        submission[['img_file', 'dot_class']].to_csv(f'{model_name.split("/")[-1]}_dot.csv', index=False)
        submission[['img_file', 'cosine_class']].to_csv(f'{model_name.split("/")[-1]}_cosine.csv', index=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
submission


,img_file,class,dot_class,cosine_class
0,64ccfecf-e451-49a8-aa3f-acf2622a9a5c.jpg,12.0,22,22
1,c6df1385-382a-4428-b41e-f2d729b90c87.jpg,22.0,22,22
2,af30e9d0-da6e-42bd-814e-c70a0c16e554.jpg,22.0,22,22
3,3fc8998e-0324-426c-8233-6b76abc7e200.jpg,NaN,7,22
4,309d1085-0d11-4411-9555-b24cc8fcee02.jpg,NaN,6,22
...,...,...,...,...
1115,43b3052a-11b6-410f-9dae-c8c60b347bb2.jpg,NaN,14,22
1116,ea11503b-de43-43e3-b46b-6a84e528217c.jpg,NaN,14,22
1117,9daf6dc7-ab1c-43fa-a733-ae9131d3686a.jpg,NaN,14,22
1118,7e548b62-5c95-478e-b61f-7a6fb0cd631b.jpg,NaN,14,22


In [ ]:
#image checking

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

metric = 'cosine_class'
target_class = 22
for i in submission.index:
  if submission[metric][i] == target_class:
    img_name = submission['img_file'][i].strip()

    img = mpimg.imread(f'/content/drive/MyDrive/extracted_images/test/images/{img_name}')
    plt.imshow(img)
    plt.axis('off')
    plt.show()